__[Open and try this file online (Colab)](https://colab.research.google.com/github/djekra/pandasklar/blob/master/jupyter/16_Compare_Series_and_DataFrames.ipynb)__

# Compare Series and DataFrames
* `compare_series`: Compares the content of two Series.
    Returns several indicators of equality.
* `compare_dataframes`: Compares the content of two DataFrames column by column. Returns several indicators of equality.
* `check_equal`: Compares the content of two DataFrames column by column.
* `compare_col_dtype`: Returns the column names of two DataFrames whose dtype differs
* `get_different_rows`: Returns the rows of two DataFrames that differ

In [1]:
# blab init
try:
    import blab
except ImportError as e:
    !pip install blab
    import blab    
startup_notebook = blab.blab_startup()
%run $startup_notebook 

blab init
environment['dropbox_path'] = /home/me/Data_Linux/Dropbox
environment['lib_path']     = /home/me/Data_Linux/Dropbox/31_Projekte/01_Python/libs
Start Time: 21:38:50


time: 393 ms


In [2]:
import pandas     as pd 
import bpyth      as bpy

# pandasklar
try:
    import pandasklar as pak 
except ImportError as e:
    !pip install pandasklar
    import pandasklar as pak   
    
# verbose
pak.Config.set('VERBOSE', True)

VERBOSE = True
--> setting verbose=True as default for all pandasklar functions

time: 303 ms


## compare_series()

In [3]:
?pak.compare_series

time: 71.3 ms


Signature: pak.compare_series(s, t, format='dict')
Docstring:
Compares the content of two Series.
Returns several indicators of equality as dict or DataFrame:
    name:    same name    
    dtype:   nearly same dtype (Float32 == Float64)
    len:     same shape        
    nnan:    same number of NaNs   
    content: same content, ignoring index and sort
    sort:    same sort order, ignoring index
    eq:      same relations index->data, ignoring sort
    
File:      ~/Data_Linux/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/compare.py
Type:      function


In [4]:
# Generate test data
s = pak.random_series( 100, 's')
s = s.apply(pak.decorate, p=0.1) # nan
s

0        0hkQ
1         NaN
2       iu3bK
3       m6LoM
4     KähnCÜo
       ...   
95    37c5avP
96     4aiy3b
97     iNwÖUö
98    JwZ8gu8
99      EcI1a
Name: rnd_string, Length: 100, dtype: object

time: 37.4 ms


In [5]:
# Generate compare data
# Play with it!

t = s.copy()
# t.name = 's' # name
# t = t[:99] # len
# t = t.apply(pak.decorate, p=0.5) # nan
#t = t.astype('object') # dtype

t[0], t[1] = t[1], t[0] 
#t = t.sort_values()



time: 16.4 ms


In [6]:
r = pak.compare_series(s,t, format='df')
r

,rnd_string
name,True
dtype,True
len,True
nnan,True
content,True
sort,False
eq,False


time: 21.6 ms


## compare_dataframes()

In [7]:
?pak.compare_dataframes

time: 14.4 ms


Signature: pak.compare_dataframes(df1, df2, reset_index=False, format='df')
Docstring:
Compares the content of two DataFrames column by column.
Returns several indicators of equality:
    name:    True, left_only or right_only. True means the column exists in both DataFrames.     
    dtype:   columns have same dtype     
    nnan:    columns have same number of NaNs   
    content: columns have same content, ignoring index and sort
    sort:    columns have same sort order, ignoring index
    eq:      columns have same relations index->data, ignoring sort    

* reset_index: Set True to ignore index and sort order or the rows.    
* format:      'DataFrame', 'Series', 'dict' or 'bool' (or abbreviations of this):
               Output format. format='DataFrame' will return detailed information.
               format='Series', 'dict' or 'bool' will return a summary.
File:      ~/Data_Linux/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/compare.py
Type:      function


In [8]:
# Generate test data
s = pak.people()
s

,first_name,age,age_class,postal_code,birthplace,secret,features,history
0,Pascal,33,30,64441,<NA>,vPNBWzZKe,"{Z, U, x}","[c, b, a]"
1,Phil,28,20,83200,Berlin,tu1keenPÜ,"{X, S, Q, 0, 5}","[A, B, C]"
2,Stephanie,27,20,79254,<NA>,OcyR1vtajd,"{r, x}","[a, b, c]"
3,Denis,32,30,60877,Berlin,AÜVu8kuWB,{r},[]
4,Annette,32,30,17713,Bremen,m9SeNjQWä,"{q, c}","[b, b, a, b]"
...,...,...,...,...,...,...,...,...
95,Beate,35,30,80018,<NA>,8pÄHzm,"{N, t, 0, w}","[A, x]"
96,Erika,36,30,63425,Bremen,nÜbC0,"{q, f, o, N, V}","[b, b, a, b]"
97,Karsten,35,30,57200,Berlin,ä3Kw5c,{T},[]
98,Anja,39,30,<NA>,Bremen,NV5FXxueQ,"{X, m, N, c}",[]


time: 114 ms


In [9]:
# Generate compare data
# Play with it!

t = s.copy()
# t.name = 's' # name
# t = t[:99] # len
# t = t.apply(pak.decorate, p=0.5) # nan
t['age'] = t.age.astype('float') # dtype

#t = t.sort_values()
#t = pak.drop_cols(t, 'age')
t['AAGE'] = 0
#t = t.sort_values(['first_name'])
#t.loc[0,'age'] = None
#t = t.head(50)
t= pak.move_cols(t,'age',-1)
t



,first_name,age_class,postal_code,birthplace,secret,features,history,AAGE,age
0,Pascal,30,64441,<NA>,vPNBWzZKe,"{Z, U, x}","[c, b, a]",0,33.0
1,Phil,20,83200,Berlin,tu1keenPÜ,"{X, S, Q, 0, 5}","[A, B, C]",0,28.0
2,Stephanie,20,79254,<NA>,OcyR1vtajd,"{r, x}","[a, b, c]",0,27.0
3,Denis,30,60877,Berlin,AÜVu8kuWB,{r},[],0,32.0
4,Annette,30,17713,Bremen,m9SeNjQWä,"{q, c}","[b, b, a, b]",0,32.0
...,...,...,...,...,...,...,...,...,...
95,Beate,30,80018,<NA>,8pÄHzm,"{N, t, 0, w}","[A, x]",0,35.0
96,Erika,30,63425,Bremen,nÜbC0,"{q, f, o, N, V}","[b, b, a, b]",0,36.0
97,Karsten,30,57200,Berlin,ä3Kw5c,{T},[],0,35.0
98,Anja,30,<NA>,Bremen,NV5FXxueQ,"{X, m, N, c}",[],0,39.0


time: 49.9 ms


In [10]:
# Output as DataFrame
pak.compare_dataframes(s,t)

,name,dtype,nnan,content,sort,eq
first_name,True,True,True,True,True,True
age_class,True,True,True,True,True,True
postal_code,True,True,True,True,True,True
birthplace,True,True,True,True,True,True
secret,True,True,True,True,True,True
features,True,True,True,True,True,True
history,True,True,True,True,True,True
age,True,False,True,True,True,True
AAGE,right_only,None,None,False,None,False
(Total),False,False,False,False,False,False


time: 44.6 ms


In [11]:
# Output as dict
pak.compare_dataframes(s,t, format='dict')

{'name': False,
 'dtype': False,
 'nnan': False,
 'content': False,
 'sort': False,
 'eq': False}

time: 47.2 ms


In [12]:
# Output as bool
pak.compare_dataframes(s,t, format='bool')

False

time: 61.1 ms


In [13]:
# This ist the same as check_equal
pak.check_equal(s,t)

False

time: 28.4 ms


## check_equal()

In [14]:
?pak.check_equal

time: 18.3 ms


Signature: pak.check_equal(obj1, obj2, reset_index=False)
Docstring:
Compares the content of two DataFrames column by column.
Two DataFrames are equal, if 
* they have the same shape
* they have the same column names
* and compare_dataframes(format='bool') is True
File:      ~/Data_Linux/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/compare.py
Type:      function


In [15]:
df1 = pak.dataframe( [ list('Babykorb'), 
                       list('abfällig'), 
                       list('Abgründe'), 
                       list('Kätzchen'), 
                       list('Landwirt'), 
                       list('lebendig'), 
                       list('Saugrohr'),       
                       list('Trugbild'),                     
                ] )

df2 = pak.dataframe( [ list('Babykorb'), 
                       list('abfällig'), 
                       list('Abgründe'), 
                       list('Kätzchen'), 
                       list('Landwirt'), 
                       list('lebendig'), 
                       list('Saugrohr'),       
                       list('Trugbild'),                     
                ] )

df1

Input rtype=('list', 'list', 'str') shape=(8, 8)
rotated=False Output rtype=('DataFrame', 'Series') shape=(8, 8)
Input rtype=('list', 'list', 'str') shape=(8, 8)
rotated=False Output rtype=('DataFrame', 'Series') shape=(8, 8)


,A,B,C,D,E,F,G,H
0,B,a,b,y,k,o,r,b
1,a,b,f,ä,l,l,i,g
2,A,b,g,r,ü,n,d,e
3,K,ä,t,z,c,h,e,n
4,L,a,n,d,w,i,r,t
5,l,e,b,e,n,d,i,g
6,S,a,u,g,r,o,h,r
7,T,r,u,g,b,i,l,d


time: 30.6 ms


In [16]:
# Initially the DataFrames are equal
assert pak.check_equal(df1, df2)

time: 43.5 ms


In [17]:
# One change >> not equal
mask = df2['A'] == 'L'
df2.loc[mask,'A'] = 'R'
assert not pak.check_equal(df1, df2)

time: 37.6 ms


In [18]:
# Change back >> equal again
mask = df2['A'] == 'R'
df2.loc[mask,'A'] = 'L'
assert pak.check_equal(df1, df2)

time: 45.7 ms


In [19]:
# change column order and row order 
df2 = pak.move_cols(df2,'D').sort_values('D')
df2

,D,A,B,C,E,F,G,H
4,d,L,a,n,w,i,r,t
5,e,l,e,b,n,d,i,g
6,g,S,a,u,r,o,h,r
7,g,T,r,u,b,i,l,d
2,r,A,b,g,ü,n,d,e
0,y,B,a,b,k,o,r,b
3,z,K,ä,t,c,h,e,n
1,ä,a,b,f,l,l,i,g


time: 22 ms


In [20]:
# still equal
assert pak.check_equal(df1, df2)

time: 37.1 ms


## compare_col_dtype()

In [21]:
?pak.compare_col_dtype

time: 20.9 ms


Signature: pak.compare_col_dtype(df1, df2)
Docstring: Returns the column names of two DataFrames whose dtype differs.
File:      ~/Data_Linux/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/compare.py
Type:      function


In [22]:
pak.compare_col_dtype(s, t)

['age']

time: 23.5 ms


## get_different_rows()

In [23]:
?pak.get_different_rows

time: 19.2 ms


Signature: pak.get_different_rows(df1, df2, indicator=True)
Docstring:
Returns the rows of two DataFrames that differ. 
Additional or missing columns are ignored.
Float columns may cause mistakes.
File:      ~/Data_Linux/Dropbox/31_Projekte/01_Python/git/pandasklar/src/pandasklar/compare.py
Type:      function


In [24]:
# Generate compare data
# Play with it!

t = s.copy()

# change one value
t.loc[0,'age'] = -1

# change order
t = t.sort_values(['first_name'])
t= pak.move_cols(t,'age',-1)

# additional or missing columns are ignored
t['AAGE'] = 0
t = pak.drop_cols(t, 'secret')

t

,first_name,age_class,postal_code,birthplace,features,history,age,AAGE
61,Ali,30,89670,Berlin,"{n, O, j, 1, H}","[a, b, c]",30,0
31,Amelie,30,59633,Berlin,"{p, U}","[A, x]",30,0
74,Amelie,20,68741,Berlin,"{o, V}","[b, b, a, b]",24,0
10,Anja,30,38889,Berlin,"{k, H}","[a, b, c]",34,0
98,Anja,30,<NA>,Bremen,"{X, m, N, c}",[],39,0
...,...,...,...,...,...,...,...,...
37,Wilhelm,30,46248,Bremen,"{d, 3, T, m, u}","[A, B, C, C]",36,0
78,Wolfgang,30,46248,<NA>,{f},"[a, b, c]",32,0
16,Wolfgang,20,91513,<NA>,"{d, n, x, p, k}","[b, b, a, b]",26,0
58,Yannick,30,87853,Berlin,"{R, 2}","[a, b, c]",34,0


time: 53.5 ms


In [25]:
pak.get_different_rows(s, t)

,first_name,age,age_class,postal_code,birthplace,secret,features,history,AAGE,_merge
0,Ali,30,30,89670,Berlin,5üFgÄö,"0 {Z, U, x}\n1 {X, S, Q, 0, 5}\n...","0 [c, b, a]\n1 [A, B, C]\n2 ...",NaN,left_only
1,Amelie,24,20,68741,Berlin,Pmzsb,"0 {Z, U, x}\n1 {X, S, Q, 0, 5}\n...","0 [c, b, a]\n1 [A, B, C]\n2 ...",NaN,left_only
2,Amelie,30,30,59633,Berlin,brwd7dS,"0 {Z, U, x}\n1 {X, S, Q, 0, 5}\n...","0 [c, b, a]\n1 [A, B, C]\n2 ...",NaN,left_only
3,Anja,21,20,54029,<NA>,ÖFmwIIeo,"0 {Z, U, x}\n1 {X, S, Q, 0, 5}\n...","0 [c, b, a]\n1 [A, B, C]\n2 ...",NaN,left_only
4,Anja,34,30,38889,Berlin,3wGfmXLdJ1,"0 {Z, U, x}\n1 {X, S, Q, 0, 5}\n...","0 [c, b, a]\n1 [A, B, C]\n2 ...",NaN,left_only
...,...,...,...,...,...,...,...,...,...,...
195,Wilhelm,36,30,46248,Bremen,<NA>,"61 {n, O, j, 1, H}\n31 {p, U}\n...","61 [a, b, c]\n31 [A, x]\n74 ...",0.0,right_only
196,Wolfgang,26,20,91513,<NA>,<NA>,"61 {n, O, j, 1, H}\n31 {p, U}\n...","61 [a, b, c]\n31 [A, x]\n74 ...",0.0,right_only
197,Wolfgang,32,30,46248,<NA>,<NA>,"61 {n, O, j, 1, H}\n31 {p, U}\n...","61 [a, b, c]\n31 [A, x]\n74 ...",0.0,right_only
198,Yannick,34,30,87853,Berlin,<NA>,"61 {n, O, j, 1, H}\n31 {p, U}\n...","61 [a, b, c]\n31 [A, x]\n74 ...",0.0,right_only


time: 170 ms


In [26]:
# does not work
#s.compare(t, align_axis=1,keep_shape=False)

time: 11.8 ms
